In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys   
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
import time
import requests
from selenium.webdriver.common.by import By
driver_path = "CHROME_DRIVER_PATH"

service = Service(driver_path)
driver = webdriver.Chrome(service=service)

url = "https://www.digikala.com/"
driver.get(url)
wait = WebDriverWait(driver, 20) 

In [3]:
search_button = wait.until(EC.element_to_be_clickable(
    (By.XPATH, "//div[contains(@class, 'lg:text-body-2') and contains(text(), 'جستجو')]")
))
search_button.click()  

In [4]:
search_box = wait.until(EC.presence_of_element_located(
    (By.XPATH, "//input[@type='text' and @placeholder='جستجو']") 
))

In [5]:
search_box.send_keys("کلاه آفتابی") 
search_box.send_keys(Keys.ENTER) 

In [6]:
clothes = []
image_urls = set()

In [7]:
image_count_limit = 50

In [ ]:

folder_path = "IMAGES_FOLDER_PATH"
os.makedirs(folder_path, exist_ok=True)

image_count = 0
while image_count < image_count_limit:
    driver.execute_script("window.scrollBy(0, 500);")
    time.sleep(1)

    products = driver.find_elements(By.XPATH, "//div[@class='flex grow relative flex-col']")

    for product in products:
        if(image_count_limit <= image_count):
            break
        try:
            image_element = product.find_element(By.XPATH, ".//img[contains(@class, 'w-full') and @width='240' and @height='240']")
            image_url = image_element.get_attribute("src")
            if(image_url in image_urls or (not image_url)):
                continue
            image_urls.add(image_url)
            price = product.find_element(By.XPATH, ".//span[@data-testid='price-final']").text
            rating = product.find_element(By.XPATH, ".//p[@class='text-body2-strong text-neutral-700']").text
            description = image_element.get_attribute("alt")
        except:
            continue

        response = requests.get(image_url)
        if response.status_code == 200:
            image_name = f"{folder_path}/image_{image_count + 1}.jpg"
            with open(image_name, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded Image {image_count + 1}: {image_name} | Price: {price} | Rating: {rating} | Description : {description}")
            image_count += 1
            clothes.append([price, rating, description, "Shoes", image_name])
        else:
            print(f"Failed to download Image {image_count + 1}: {image_url}")


driver.quit()

In [ ]:
clothes = pd.DataFrame(clothes, columns=['price', 'rating', 'description', 'category', 'image_path'])

In [ ]:
clothes

In [ ]:
clothes.to_csv("DATAFRAME_CSV_PATH")